# Handling openfoam Fields. 

When running the code, make sure that you either: 
1. Know the name of the project (that was created by hera) and supply it to the getToolkit code

**or**

2. work in a directory where the project was defined (caseCondiguration.json). 

it is in a project that was created by hera. 
That is, that you have the `caseConfiguration.json` in your directory (if you don't want to supply a project name) 

## Initialization

Get hera and the toolkit. 

In [20]:
from hera import toolkitHome
tk = toolkitHome.getToolkit(toolkitName=toolkitHome.SIMULATIONS_OPENFOAM,projectName="DOCUMENTATION")

Note that the projectName is redundent if you have the caseConfiguration in the directory of the code. 

## Reading the mesh 

We can read the cell centers of the mesh. If the case is paralll, the code reads the mesh centers in the parallel case. 
That is, it provides the map of the local indexing and the processors. 

To do so, the program runs the writeCellCenters application, if the center field does not exist. 
The code also computes the centers of the boundary faces. 

In [8]:
caseMesh= tk.getMesh("Indoor_1")

Parallel processing using SYSTEMOPENMPI with 16 processors
Executing: /usr/bin/mpirun -np 16 -x FOAM_SETTINGS /opt/openfoam10/bin/foamExec -prefix /opt postProcess -func writeCellCentres -parallel > log 2>&1


In [11]:
meshDataFrame = caseMesh.getDataFrame()
meshDataFrame

,processorIndex,Cx,Cy,Cz,processor,region,boundary,type
0,0,-1.586248,-2.377223,0.091947,0,internalField,NaN,NaN
1,1,-1.451249,-2.377223,0.091948,0,internalField,NaN,NaN
2,2,-1.586231,-2.258122,0.097170,0,internalField,NaN,NaN
3,3,-1.451239,-2.258116,0.097175,0,internalField,NaN,NaN
4,4,-1.586203,-2.124986,0.101950,0,internalField,NaN,NaN
...,...,...,...,...,...,...,...,...
9705,542,2.260456,-1.653748,0.000000,9,boundaryField,floor,calculated
9706,543,1.856234,-1.586247,0.000000,9,boundaryField,floor,calculated
9707,544,1.991238,-1.586248,0.000000,9,boundaryField,floor,calculated
9708,545,2.126434,-1.586248,0.000000,9,boundaryField,floor,calculated


The processor index is the local index (e.g. number of the node) of the cell in each processor. 


## Reading the field

Reading an empty field (that will include all the boundries, and the internal processor boundaries) is done by

In [13]:
newField = tk.OFObjectHome.getEmptyFieldFromCase("U",tk.FLOWTYPE_INCOMPRESSIBLE,"Indoor_1")

The Flow type (compressible or incompressible) is important to guess the units of the field. 
The last parameter is the case name. 

The units of the field are determined by the name. If the field does not pre-exist, it has to be registered in in OFObjectHome using the method addFieldDefinitions. 

Reading an exsiting field 

In [14]:
fieldU = tk.OFObjectHome.readFieldFromCase("U",tk.FLOWTYPE_INCOMPRESSIBLE,"Indoor_1")

In [16]:
fieldU.getDataFrame()

,processorIndex,Ux,Uy,Uz,processor,region,boundary,type
0,0,0.0,0.0,0.0,0,internalField,NaN,NaN
1,0,0.0,0.0,0.0,0,boundaryField,inlet1,surfaceNormalFixedValue
2,0,NaN,NaN,NaN,0,boundaryField,Walls,noSlip
3,0,NaN,NaN,NaN,0,boundaryField,ceiling,noSlip
4,0,NaN,NaN,NaN,0,boundaryField,door,zeroGradient
...,...,...,...,...,...,...,...,...
3,0,NaN,NaN,NaN,9,boundaryField,ceiling,noSlip
4,0,NaN,NaN,NaN,9,boundaryField,door,zeroGradient
5,0,NaN,NaN,NaN,9,boundaryField,floor,noSlip
6,0,0.0,0.0,0.0,9,boundaryField,inlet2,surfaceNormalFixedValue


## The field as dataframe. 

You can view the field as pandas.dataframe

In [7]:
dataFrame = field.getDataFrame()
dataFrame

,processorIndex,Ux,Uy,Uz,processor,region,boundary,type
0,0,0.0,0.0,0.0,0,internalField,NaN,NaN
1,0,NaN,NaN,NaN,0,boundaryField,Walls,zeroGradient
2,0,NaN,NaN,NaN,0,boundaryField,ceiling,zeroGradient
3,0,NaN,NaN,NaN,0,boundaryField,inlet2,zeroGradient
4,0,NaN,NaN,NaN,0,boundaryField,outlet,zeroGradient
...,...,...,...,...,...,...,...,...
3,0,NaN,NaN,NaN,9,boundaryField,inlet2,zeroGradient
4,0,NaN,NaN,NaN,9,boundaryField,outlet,zeroGradient
5,0,NaN,NaN,NaN,9,boundaryField,inlet1,zeroGradient
6,0,NaN,NaN,NaN,9,boundaryField,door,zeroGradient


The coordinates of the node are given by the mesh. However, if the code is defined with uniform, then there will be only one line 
of the internalField. Currently, there are no automated functions to expand it to all the cells (except for outer joining the dataframe of the field and the dataframe of the mesh). 

## Setting the data

Setting the field can be done in two ways: 

1. **Directly**: directly through the pyFOAM interface.

In [17]:
newField.internalField('processor0')

'uniform (0 0 0)'

In [ ]:
or the boundary fields

In [19]:
newField.boundaryField('processor0')

{'Walls': {'type': 'zeroGradient'},
 'ceiling': {'type': 'zeroGradient'},
 'inlet2': {'type': 'zeroGradient'},
 'outlet': {'type': 'zeroGradient'},
 'inlet1': {'type': 'zeroGradient'},
 'door': {'type': 'zeroGradient'},
 'floor': {'type': 'zeroGradient'},
 'procBoundary0to1': {'type': 'processor'},
 'procBoundary0to2': {'type': 'processor'},
 'procBoundary0to3': {'type': 'processor'},
 'procBoundary0to8': {'type': 'processor'}}

2. **Using dataframe**: Updating the dataframe (that was obtained by getDataframe) and then 
set it using 


In [ ]:
newField.setFieldFromDataFrame(dataFrame)

## Wrting the field 

To write the field, use the 

In [ ]:
newField.writeToCase("caseFile","timeStep")